## Exercise 5: Geospatial wrangling and making maps

Skills: 
* More geospatial practice building on earlier skills
* Make a map with `geopandas`

References: 
* https://docs.calitp.org/data-infra/analytics_new_analysts/02-data-analysis-intermediate.html
* https://docs.calitp.org/data-infra/analytics_tools/python_libraries.html

In [ ]:
import geopandas as gpd
import intake
import os
import pandas as pd
import shapely

os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp_data_analysis.tables import tbls
from siuba import *

# Hint: if this doesn't import: refer to docs for correctly import
# cd into _shared_utils folder, run the make setup_env command
import shared_utils

## Research Question

What's the average number of trips per stop by operators in southern California? Show visualizations at the operator and county-level.
<br>**Geographic scope:** southern California counties
<br>**Deliverables:** chart(s) and map(s) showing metrics comparing across counties and also across operators. Make these visualizations using function(s).

### Prep data

* Use the same query, but grab a different set of operators. These are in southern California, so the map should zoom in counties ranging from LA to SD.
* *Hint*: for some counties, there are multiple operators. Make sure the average stop events per stop by counties is the weighted average.
* Use the same [shapefile for CA counties](https://gis.data.ca.gov/datasets/CALFIRE-Forestry::california-county-boundaries/explore?location=37.246136%2C-119.002032%2C6.12) as in Exercise 4.
* Join the data and only keep counties that have bus stops.
* If you cannot connect to the warehouse, use this dict to map feed_keys to names.
    ```
    feed_keys_to_names_dict = {
        "71d91d70ad6c07b1f9b0a618ffceef93": "Alhambra Schedule",
        "a7ba6f075198e9bf9152fab6c7faf0f6": "San Diego Schedule",
        "4f77ef02b983eccc0869c7540f98a7d0": "Big Blue Bus Schedule"
        "ae93a53469371fb3f9059d2097f66842": "OmniTrans Schedule",
        "180d48eb03829594478082dca5782ccd": "Culver City Schedule"
    }
    ```

In [ ]:
feeds_to_names = shared_utils.gtfs_utils_v2.schedule_daily_feed_to_organization(
    selected_date = "2022-06-01",
    get_df = True
)[["feed_key", "name"]].drop_duplicates()

In [ ]:
OPERATORS = [
    "Alhambra Schedule", 
    "San Diego Schedule",
    "Big Blue Bus Schedule",
    "Culver City Schedule",
    "OmniTrans Schedule",
    "OCTA Schedule"
]

SUBSET_FEEDS = feeds_to_names[
    feeds_to_names.name.isin(OPERATORS)
].feed_key.tolist()

In [ ]:
stops = (
    tbls.mart_gtfs.fct_daily_scheduled_stops()
    >> filter(_.feed_key.isin(SUBSET_FEEDS))
    >> filter(_.activity_date == "2022-06-01")
    >> select(_.feed_key, 
              _.stop_id, _.pt_geom)
    >> collect()
)

Check the type of `stops`. Is it a pandas df or geopandas gdf?

In [ ]:
# Turn stops into a gdf
geom = [shapely.wkt.loads(x) for x in stops.pt_geom]

stops = gpd.GeoDataFrame(
    stops, 
    geometry=geom, 
    crs="EPSG:4326"
).drop(columns="pt_geom")

Check the type of `stops`. Is it a pandas df or geopandas gdf?

What is the CRS and geometry column name?

### Bring in a new table from BigQuery

* In `mart_gtfs`, bring in the table called `fct_daily_scheduled_stops` for the subset of feeds defined above.
* Modify the snippet below to:
   * filter for the subset of operators
   * only keep columns: `feed_key`, `stop_id`, `stop_event_count`

In [ ]:
stop_counts = (
    tbls.mart_gtfs.fct_daily_scheduled_stops()
    >> filter(_.activity_date == "2022-06-01")
)

### Aggregate
* Write a function to aggregate to the operator level or county level, add new columns for desired metrics.
* Merge in CA shapefile to get a gdf.
* Add another `geometry` column, called `centroid`, and grab the county's centroid.
* Refer to [docs](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.set_geometry.html) to see how to pick which column to use as the `geometry` for the gdf, since technically, a gdf can handle multiple geometry columns.

### Step by Step
* Merge the 2 dataframes (`stops` and `stop_counts`) so that for every stop, you have the stop counts and the point geometry.
* Put the gdf on the left so that your resulting df is a gdf.
* Check the `type()` and make sure your merged df is a gdf.

Aggregate the merged stops gdf to the operator level.

Find the number of stops an operator has and the total number of stop events that occurred for that operator.

Do a spatial join of stops to CA counties.

This answers the question: which county does this stop fall in? 
```
stops_in_counties = gpd.sjoin(
   stops,
   ca_counties,
   how = "inner",
   predicate = "intersects"
)
```

Take a look at the result. What new column is added? What is the geometry column? Is it point or line or polygon?

Only keep the counties that have stops with it. Adjust the spatial join `how=` parameter to do this.

Aggregate the spatial join result to the county level. 

For each county, calculate the number of operators, stops, and stop events. 

Calculate the number of stop events per stop for each county. Add this as a new column.

### Visualizations
* Make one chart for comparing trips per stop by operators, and another chart for comparing it by counties. Use a function to do this.
* Make 1 map for comparing trips per stop by counties. Use `gdf.explore()` to do this.
* Visualizations should follow the Cal-ITP style guide: [styleguide example notebook](https://github.com/cal-itp/data-analyses/blob/main/starter_kit/style-guide-examples.ipynb)
* More on `folium` and `ipyleaflet`: https://github.com/jorisvandenbossche/geopandas-tutorial/blob/master/05-more-on-visualization.ipynb

In [ ]:
# To add styleguide
from calitp_data_analysis import styleguide
from calitp_data_analysis import calitp_color_palette as cp

Make a chart and a map of total stop events by county.

Make a chart and a map of stop events per stop by county.

Use a Markdown cell and write how you would summarize and interpret the visualizations.